In [ ]:
%pip install mtcnn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
import pandas as pd

import cv2
from mtcnn import MTCNN
import matplotlib.pyplot as plt

In [1]:
FRAMES_MAIN_PATH = '/content/drive/MyDrive/Frames'
SAVE_MAIN_PATH = '/content/drive/MyDrive/Detected_Faces'

persons = ['Karolina','Kacper','Joanna','Mirek','Sylwia','Łukasz','Emilia','Mariusz','Ania'] ## classes

In [ ]:
def show_image(rgb_image, x1, y1, x2, y2):
    '''
    Show image with rectangle around detected face

    params:
        rgb_image: np.array - image in RGB format
        x1, y1, x2, y2: int - coordinates of the rectangle
    '''
    cv2.rectangle(rgb_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    plt.imshow(rgb_image)
    plt.axis('off')
    plt.show()

def detect_face(folder_name:str, data_type:str, show = False):
    '''
    Extract faces from single person folder

    params:
        folder_name: str - name of the folder with frames
        data_type: str - type of the data (no_covering, sunglasses)
        show: bool - if True show the image with detected face

    return:
        extracted_faces: list - list of extracted faces
    '''
    path_to_data = os.path.join(FRAMES_MAIN_PATH, data_type, folder_name)
    path_to_save = os.path.join(SAVE_MAIN_PATH, data_type, folder_name)

    os.makedirs(path_to_save, exist_ok=True)

    extracted_faces = []
    detector = MTCNN()
    saved_count = 0

    for image_path in os.listdir(path_to_data):
        image = cv2.imread(os.path.join(path_to_data, image_path))
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        faces = detector.detect_faces(rgb_image)

        if len(faces) == 0:
            print("No faces for: ",os.path.join(path_to_data, image_path))
            continue

        x1, y1, width, height = faces[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height

        if show == True:
            show_image(rgb_image, x1, y1, x2, y2)

        face_image = rgb_image[y1:y2, x1:x2]
        extracted_faces.append(face_image)

        save_path = os.path.join(path_to_save,f"cropped_{saved_count:04d}.jpg")
        cv2.imwrite(save_path, face_image)
        saved_count +=1
        print(f'SAVED data:   {save_path}')

    return extracted_faces

##### Detecting faces of people without any covering

In [ ]:
for name in persons:
    for i in range(1,4):
        extracted_faces = detect_face(f'{name}/{name}_{i}', 'no_covering', show = False)

##### Detecting faces of people wearing sunglasses

In [ ]:
for name in persons:
    for i in range(1,4):
        extracted_faces = detect_face(f'{name}/{name}_{i}', 'sunglasses', show = False)